[View in Colaboratory](https://colab.research.google.com/github/BUPT/awesome-chatbot/blob/master/code/CAIC_11_Capsule_experiment_on_Mnist.ipynb)

# CAIC #11 Capsule experiment in Mnist
## Inference:
- https://kexue.fm/archives/4819
- https://github.com/bojone/Capsule/blob/master/capsule_test.py

## Presention by:
- 824zzy

## Keras版本Capsule模型的实现

In [2]:
import numpy as np
from keras import activations
from keras import utils
from keras.datasets import mnist
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.engine.topology import Layer

Using TensorFlow backend.


In [0]:
# 论文原文是１，实验结果表明０.５效果更好。可以继续实验
def squash(x, axis=-1):
    # 加一个epsilon，保证
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x

In [0]:
# 定义自己的softmax函数
# K.max: a trick when computing softmax avoiding overflow
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)

In [0]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, share_weights=True, activation='squash', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == "squash":
            self.activation = squash
        else:
            self.activation = activation.get(activation)
           
    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)
            
    def call(self, u_vecs):
        print('origin u_vecs shape:', u_vecs.shape)
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])
        print('conved u_vecs shape:', u_hat_vecs.shape)
        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        print('reshaped u_vecs shape:', u_hat_vecs.shape)
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        print('permuted u_vecs shape:', u_hat_vecs.shape)
        b = K.zeros_like(u_hat_vecs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            print("c shape", c.shape)
            s = K.batch_dot(c, u_hat_vecs, [2, 2])
            print("s shape", s.shape)
            print("b shape", b.shape)
            if K.backend() == 'theano':
                s = K.sum(s, axis=1)
            if i < self.routings - 1:
                s = K.l2_normalize(s, -1)
                b = K.batch_dot(s, u_hat_vecs, [2, 3])
                if K.backend() == 'theano':
                    b = K.sum(b, axis=1)
                    
        return self.activation(s)
                
            
    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

## Capsule模型在Mnist分类任务上的实验：
实验细节如下：
1. **将一个多分类的任务转换为多个二分类的任务**
2. 测试的图片并不是原始的测试集，而是随机将两张测试集的图片拼在一起，看模型是否能预测出两个数字和两个数字的顺序。
3. 测试可以继续加大难度，测试
    1. 普通结构：CNN + Pooling
    2. Capsule结构：CNN + Capsule
    在mnist数字**序列**的预测准确率


### 构造实验数据

In [6]:
# 准备训练数据
batch_size = 128
num_classes = 10
img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 10)
(10000, 10)


In [7]:
# 自定义测试样本
# 对测试集重新排序
idx = range(len(x_test))
np.random.shuffle(idx)
X_test = np.concatenate([x_test, x_test[idx]], 1)
print "拼接两张图片后的测试集X_test的形状：" + str(X_test.shape)
Y_test = np.vstack([y_test.argmax(1), y_test[idx].argmax(1)]).T
print "拼接两张图片后的测试集Y_test的形状：" + str(Y_test.shape)
X_test = X_test[Y_test[:,0] != Y_test[:,1]] #确保两个数字不一样
Y_test = Y_test[Y_test[:,0] != Y_test[:,1]]

print "经过简单处理后的拼接两张图片后的测试集X_test的形状：" + str(X_test.shape)
print "经过简单处理后的拼接两张图片后的测试集Y_test的形状：" + str(Y_test.shape)

print "排序前的Y_test:"
print Y_test
Y_test.sort(axis=1) #排一下序，因为只比较集合，不比较顺序, todo:对这个步骤保留意见
print "排序后的Y_test:"
print Y_test

拼接两张图片后的测试集X_test的形状：(10000, 56, 28, 1)
拼接两张图片后的测试集Y_test的形状：(10000, 2)
经过简单处理后的拼接两张图片后的测试集X_test的形状：(8966, 56, 28, 1)
经过简单处理后的拼接两张图片后的测试集Y_test的形状：(8966, 2)
排序前的Y_test:
[[7 6]
 [2 3]
 [1 7]
 ...
 [4 7]
 [5 8]
 [6 2]]
排序后的Y_test:
[[6 7]
 [2 3]
 [1 7]
 ...
 [4 7]
 [5 8]
 [2 6]]


### 普通CNN模型及其实验结果

In [8]:
# 普通CNN分类模型
input_image = Input(shape=(None,None,1))
cnn = Conv2D(64, (3, 3), activation='relu')(input_image)
cnn = Conv2D(64, (3, 3), activation='relu')(cnn)
cnn = AveragePooling2D((2,2))(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = GlobalAveragePooling2D()(cnn)
dense = Dense(128, activation='relu')(cnn)
output = Dense(10, activation='sigmoid')(dense)

model = Model(inputs=input_image, outputs=output)
model.compile(loss=lambda y_true,y_pred: y_true*K.relu(0.9-y_pred)**2 + 0.25*(1-y_true)*K.relu(y_pred-0.1)**2,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 128)   147584    
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)               0         
__________

In [9]:
Y_pred = model.predict(X_test) #模型预测
print "Y_pred's shape:" + str(Y_pred.shape)
print "original Y_pred is: "
print Y_pred

greater = np.sort(Y_pred, axis=1)[:, -2] > 0.5 #判断预测结果是否大于0.5
Y_pred = Y_pred.argsort()[:,-2:]
Y_pred.sort(axis=1)
print "sorted Y_pred is:"
print(Y_pred)

acc = 1.*(np.prod(Y_pred == Y_test, axis=1)).sum()/len(X_test)
print u'CNN+Pooling，不考虑置信度的准确率为：%s'%acc
acc = 1.*(np.prod(Y_pred == Y_test, axis=1)*greater).sum()/len(X_test)
print u'CNN+Pooling，考虑置信度的准确率为：%s'%acc

Y_pred's shape:(8966, 10)
original Y_pred is: 
[[3.5898297e-03 6.3792076e-03 2.4300043e-01 ... 1.9211645e-01
  5.8910460e-04 9.3354225e-02]
 [4.5615687e-07 1.0364586e-02 1.2951662e-01 ... 6.7579263e-04
  6.2574272e-06 6.8845187e-04]
 [2.5748779e-05 1.5192473e-01 5.3542849e-02 ... 7.2596663e-01
  6.6961540e-05 4.0533193e-03]
 ...
 [4.6497542e-05 7.2127208e-03 4.0396877e-02 ... 3.0296931e-01
  2.2483692e-03 6.4190403e-02]
 [2.5548441e-06 2.0745485e-06 4.2922608e-07 ... 1.3297800e-06
  3.1802163e-02 3.3478562e-03]
 [1.7010028e-04 3.2451584e-03 4.2940500e-01 ... 1.1197163e-02
  1.3958442e-04 3.9744778e-03]]
sorted Y_pred is:
[[2 6]
 [3 5]
 [1 7]
 ...
 [4 7]
 [5 8]
 [2 6]]
CNN+Pooling，不考虑置信度的准确率为：0.348315859915
CNN+Pooling，考虑置信度的准确率为：0.080637965648


### Capsule模型及其实验结果

In [10]:
#搭建CNN+Capsule分类模型
input_image = Input(shape=(None,None,1))
cnn = Conv2D(64, (3, 3), activation='relu')(input_image)
cnn = Conv2D(64, (3, 3), activation='relu')(cnn)
cnn = AveragePooling2D((2,2))(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Conv2D(128, (3, 3), activation='relu')(cnn)
cnn = Reshape((-1, 128))(cnn)
capsule = Capsule(10, 16, 3, True)(cnn)
output = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)), output_shape=(10,))(capsule)

model = Model(inputs=input_image, outputs=output)
model.compile(loss=lambda y_true,y_pred: y_true*K.relu(0.9-y_pred)**2 + 0.25*(1-y_true)*K.relu(y_pred-0.1)**2,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))


('origin u_vecs shape:', TensorShape([Dimension(None), Dimension(None), Dimension(128)]))
('conved u_vecs shape:', TensorShape([Dimension(None), Dimension(None), Dimension(160)]))
('reshaped u_vecs shape:', TensorShape([Dimension(None), Dimension(None), Dimension(10), Dimension(16)]))
('permuted u_vecs shape:', TensorShape([Dimension(None), Dimension(10), Dimension(None), Dimension(16)]))
('c shape', TensorShape([Dimension(None), Dimension(10), Dimension(None)]))
('s shape', TensorShape([Dimension(None), Dimension(10), Dimension(16)]))
('b shape', TensorShape([Dimension(None), Dimension(10), Dimension(None)]))
('c shape', TensorShape([Dimension(None), Dimension(10), Dimension(None)]))
('s shape', TensorShape([Dimension(None), Dimension(10), Dimension(16)]))
('b shape', TensorShape([Dimension(None), Dimension(10), Dimension(None)]))
('c shape', TensorShape([Dimension(None), Dimension(10), Dimension(None)]))
('s shape', TensorShape([Dimension(None), Dimension(10), Dimension(16)]))
('b sh

In [11]:
Y_pred = model.predict(X_test) #用模型进行预测
greater = np.sort(Y_pred, axis=1)[:,-2] > 0.5 #判断预测结果是否大于0.5
Y_pred = Y_pred.argsort()[:,-2:] #取最高分数的两个类别
Y_pred.sort(axis=1) #排序，因为只比较集合

acc = 1.*(np.prod(Y_pred == Y_test, axis=1)).sum()/len(X_test)
print u'CNN+Capsule，不考虑置信度的准确率为：%s'%acc
acc = 1.*(np.prod(Y_pred == Y_test, axis=1)*greater).sum()/len(X_test)
print u'CNN+Capsule，考虑置信度的准确率为：%s'%acc

CNN+Capsule，不考虑置信度的准确率为：0.961744367611
CNN+Capsule，考虑置信度的准确率为：0.961744367611
